In [1]:
#import pandas as pd
import numpy as np
from multiprocessing import Process, Pipe
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from os import cpu_count
import sys

In [2]:
try:
    with open('./input/inputs.npy',"rb") as f:
        inputs = np.load(f)
    with open('./input/outputs.npy',"rb") as f:
        outputs = np.load(f)
        
except:
    data = np.genfromtxt('./input/train.csv', delimiter=',',skip_header=1,dtype=np.uint8)
    inputs = data[:,1:]
    outputs = data[:,0]
    np.save('./input/inputs.npy',inputs)
    np.save('./input/outputs.npy',outputs)
print(f'Input:\nShape:{inputs.shape}\n{inputs[:5,:5]}\n')
print(f'Output:\nShape:{outputs.shape}\n{outputs}\n')

Input:
Shape:(42000, 784)
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Output:
Shape:(42000,)
[1 0 1 ... 7 6 9]



In [3]:
inputs = normalize(inputs)

X_train, X_test, y_train, y_test = train_test_split(inputs,outputs,test_size = 0.95)

X_train = X_train.T
y_train_hot = np.eye(10)[y_train].astype(np.uint8).T

l_dim = np.array([X_train.shape[0],397,10],dtype = np.uint16)
w = [int]*(len(l_dim)+1)
b = [int]*(len(l_dim)+1)

In [7]:
def sigmoid(X):
    return np.exp(X)/(1 + np.exp(X))

def sigprime(X):
    return np.exp(X)/np.power(1 + np.exp(X),2)

def initialize_wb(LAYERS):
    for i in range(0,len(LAYERS)-1):
        w[i] = np.array(np.random.random((LAYERS[i+1],LAYERS[i])), dtype=np.float128)
        b[i]  = np.zeros((LAYERS[i+1],1), dtype=np.float16)
    return w, b

def feedforward(W,X,B):
    return normalize(sigmoid(np.dot(W,X) + B))

def cost(PRED,TRUE):
    return 1/2*np.sum(np.power(np.subtract(PRED,TRUE),2))

In [8]:
w, b = initialize_wb(l_dim)
normalize(feedforward(w[0],X_train,b[0]))

array([[0.02181335, 0.02181896, 0.02182314, ..., 0.02188905, 0.02193719,
        0.02166867],
       [0.02181253, 0.02183327, 0.0218368 , ..., 0.02188983, 0.02194885,
        0.02175236],
       [0.02181333, 0.02184195, 0.02183674, ..., 0.02190956, 0.02199049,
        0.02166524],
       ...,
       [0.02178065, 0.02182986, 0.02182803, ..., 0.02182319, 0.02194877,
        0.02173285],
       [0.02180153, 0.02183103, 0.02180938, ..., 0.02185652, 0.02194307,
        0.02167154],
       [0.02183943, 0.02177822, 0.02179939, ..., 0.02187886, 0.0219614 ,
        0.02156362]])

In [9]:
w, b = initialize_wb(l_dim)
layer = X_train
for i in range(0,len(l_dim)-1):
    sys.stdout.write(f'{i}\r')
    newlayer = feedforward(w[i],layer,b[i])
    print(newlayer.shape)
    layer = newlayer
print(cost(layer,y_train_hot))

(397, 2100)
(10, 2100)
1009.1737997032033


In [6]:
np.sum(np.power(np.subtract(layer,y_train_hot),2))

18900.0

1.0